<a href="https://colab.research.google.com/github/khnhenriette/ProjectADL/blob/math-medium/notebooks/medium_fine_tune_math_hpo_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hyper-Parameter optimization: Fine tune gpt2-medium for basic math tasks

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets
!pip install torch
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 23.1 MB/s eta 0:00:0

Use the dataset math_dataset.json that includes 20000 simple math examples of the form "89 minus 84 equals 5" using addition, subtraction, multiplication and division -- ensure dataset is uploaded to Google Colab before running

In [3]:
import pandas as pd
from datasets import Dataset

# Load the CSV dataset
df = pd.read_csv('math_dataset.csv')  # Update this path if using Google Drive

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Shuffle the dataset
dataset = dataset.shuffle(seed=42)  # Ensure reproducibility with a fixed seed

# Split into training (5000 examples) and the rest
train_dataset = dataset.select(range(5000))
remaining_dataset = dataset.select(range(5000, len(dataset)))

# Split the remaining dataset into validation (500 examples) and test (500 examples)
eval_dataset = remaining_dataset.select(range(500))
test_dataset = remaining_dataset.select(range(500, 1000))

# Print dataset sizes
print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(eval_dataset)}")
print(f"Test examples: {len(test_dataset)}")


Training examples: 5000
Validation examples: 500
Test examples: 500


In [4]:
print(train_dataset[4])

{'math_problem': '13 minus 26 equals -13'}


### Use the Hugging Face Trainer to fine-tune the available gpt2-medium to perform better on the simple math tasks

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# Load GPT-2 Medium tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Start by finding right max_length for padding


In [6]:
# Check the longest sequence in your dataset
max_tokenized_length = max(len(tokenizer(example)['input_ids']) for example in dataset['math_problem'])
print(f"Maximum tokenized length: {max_tokenized_length}")


Maximum tokenized length: 8


In [7]:

# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token

# choose max_length slightly higher than longest sequence in dataset
max_length = 32

# Tokenize the dataset
def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["math_problem"],
        truncation=True,
        padding="max_length",
        max_length=max_length
    )
    # Labels are the same as input_ids for causal language modeling
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


In [8]:
# ensure padding and masking are done correctly

testing = tokenizer("3 plus 65 equals 68", padding="max_length", max_length=10)
print(testing)


{'input_ids': [18, 5556, 6135, 21767, 8257, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]}


In [13]:
# tokenize dataset

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
# mask eos tokens in labels to avoid their prediction in final outcome

def mask_eos_in_labels(example):
    example["labels"] = [token if token != tokenizer.eos_token_id else -100 for token in example["labels"]]
    return example

train_dataset = train_dataset.map(mask_eos_in_labels)
eval_dataset = eval_dataset.map(mask_eos_in_labels)
test_dataset = test_dataset.map(mask_eos_in_labels)


print(train_dataset[0])
print(eval_dataset[0])
print(test_dataset[0])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'math_problem': '77 times 26 equals 2002', 'input_ids': [3324, 1661, 2608, 21767, 6244, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [3324, 1661, 2608, 21767, 6244, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]}
{'math_problem': '41 divided by 94 equals 0.44', 'input_ids': [3901, 9086, 416, 10048, 21767, 657, 13, 2598, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label

Train with hyperparameter optimization using optuna

In [11]:
import optuna
from transformers import TrainingArguments, Trainer

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_epochs = trial.suggest_int("num_epochs", 1, 5)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./gpt2_finetuned_optuna",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_epochs,
        save_steps=500,
        save_total_limit=2,
        logging_dir="./logs",
        logging_steps=50,
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    # Train the model and evaluate on validation set
    trainer.train()
    eval_results = trainer.evaluate()

    # Use validation loss as the optimization metric
    return eval_results["eval_loss"]

# Run the hyperparameter search
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10, show_progress_bar=True)
print("Tested learning rates:", [trial.params["learning_rate"] for trial in study.trials])

# Best hyperparameters
print("Best hyperparameters according to HPO:", study.best_params)



[I 2024-12-06 18:32:27,916] A new study created in memory with name: no-name-08ef3c77-d776-4a1c-b497-702f2c8cfd8c


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-09a2d6589644>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

Epoch,Training Loss,Validation Loss
1,0.682800,2.738264
2,0.874700,2.657483
3,1.035200,2.545393
4,1.076600,2.488992
5,1.069800,2.479054


[I 2024-12-06 19:33:33,955] Trial 8 finished with value: 2.4790537357330322 and parameters: {'learning_rate': 5.102478663560619e-06, 'batch_size': 4, 'num_epochs': 5}. Best is trial 1 with value: 1.9355098009109497.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-09a2d6589644>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.797600,2.679770


[I 2024-12-06 19:34:56,890] Trial 9 finished with value: 2.679769515991211 and parameters: {'learning_rate': 2.3325870504045444e-06, 'batch_size': 8, 'num_epochs': 1}. Best is trial 1 with value: 1.9355098009109497.
Tested learning rates: [1.1793664936074914e-05, 5.2451161530399926e-05, 5.48231492011936e-05, 5.300314773918238e-06, 3.742461233896312e-05, 2.5870143142934004e-06, 6.65174239816731e-06, 3.9060403786337064e-05, 5.102478663560619e-06, 2.3325870504045444e-06]
Best hyperparameters according to HPO: {'learning_rate': 5.2451161530399926e-05, 'batch_size': 4, 'num_epochs': 5}


In [12]:
stop_execution()

NameError: name 'stop_execution' is not defined

Check the HPO outcomes and choose the best hyperparameters: the code above does not take into account overfitting so make sure to choose well!

In [ ]:
best_params = {'learning_rate': , 'batch_size': , 'num_epochs': }

In [15]:
# fine tune using best HPs

best_hyperparams = study.best_params

training_args = TrainingArguments(
    output_dir="./gpt2_finetuned_hpo_small",
    evaluation_strategy="epoch",
    learning_rate=best_hyperparams["learning_rate"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    num_train_epochs=best_hyperparams["num_epochs"],
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-c89a488280c9>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.993500,2.667672
2,1.125200,2.459021
3,1.183800,2.457612
4,1.131300,2.478989
5,1.054900,2.514518


TrainOutput(global_step=6250, training_loss=1.0241088479614258, metrics={'train_runtime': 648.2968, 'train_samples_per_second': 38.563, 'train_steps_per_second': 9.641, 'total_flos': 1451094835200000.0, 'train_loss': 1.0241088479614258, 'epoch': 5.0})

In [16]:
# verify tokens are set consistently

print(f"Pad token ID: {tokenizer.pad_token_id}")
print(f"EOS token ID: {tokenizer.eos_token_id}")


Pad token ID: 50256
EOS token ID: 50256


In [17]:
# Try out the model

# Function to predict the next token
def predict_next_token(input_text):
    # Tokenize and move input to GPU
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    # Generate only the next token
    outputs = model.generate(
        **inputs,
        max_new_tokens=1,  # Generate only one token
        pad_token_id=tokenizer.pad_token_id,  # Ensure padding is handled
        eos_token_id=tokenizer.eos_token_id  # Set EOS token explicitly
    )
    # Decode and return the output
    return tokenizer.decode(outputs[0])

# Test cases
input_texts = [
    "76 plus 24 equals",
    "80 minus 24 equals",
    "4 times 10 equals",
    "40 divided by 5 equals"
]

# Generate and print the next token for each input
for input_text in input_texts:
    next_token = predict_next_token(input_text)
    print(f"Input: {input_text}\nNext Token: {next_token}\n")


Input: 76 plus 24 equals
Next Token: 76 plus 24 equals 100

Input: 80 minus 24 equals
Next Token: 80 minus 24 equals 52

Input: 4 times 10 equals
Next Token: 4 times 10 equals 20

Input: 40 divided by 5 equals
Next Token: 40 divided by 5 equals 10



In [18]:
# get performance estiamtion in terms of accuracy

from sklearn.metrics import accuracy_score

# Load tokenized test dataset (example format; replace with your actual data)
# Each entry is a dictionary with 'input_ids' and 'labels', and padding tokens are set to -100 in 'labels'.
test_data = test_dataset
print(test_data[0])

# Prepare data for evaluation
prompts = []
correct_token_dec = []

for item in test_data:
    # Find the last non-padding token (label not equal to -100)
    last_valid_index = len(item['labels']) - 1 - item['labels'][::-1].index(-100)
    prompts.append(item['input_ids'][:last_valid_index])  # Input up to but not including the last token
    correct_token_dec.append(item['labels'][tokenizer.decode(last_valid_index)])  # The last valid label decoded into output

# Evaluate accuracy for predicting the last valid token
predicted_tokens = []
for prompt in prompts:
    next_token = predict_next_token(prompt)
    predicted_tokens.append(next_token)

# check everything is stored correctly -- should be two numbers (ideally the same)
print(correct_token_dec[0])
print(predicted_tokens[0])

# Calculate accuracy
accuracy = accuracy_score(correct_token_dec, predicted_tokens)
accuracy_percentage = accuracy * 100

# Display results
print(f"Accuracy for predicting the last valid token: {accuracy_percentage:.2f}%")



{'math_problem': '29 plus 3 equals 32', 'input_ids': [1959, 5556, 513, 21767, 3933, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [1959, 5556, 513, 21767, 3933, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]}


TypeError: list indices must be integers or slices, not str

Save the fine-tuned model for future use

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./gpt2_finetuned_hpo_small")
tokenizer.save_pretrained("./gpt2_finetuned_hpo_small")


In [ ]:
!zip -r gpt2_finetuned_hpo_small.zip ./gpt2_finetuned_hpo_small


In [ ]:
!mv gpt2_finetuned_hpo_small.zip /content/drive/MyDrive/
